In [1]:
%run create_4LAC_for_AGN_Pop.ipynb

Total number of 4LAC entries found in the 4FGL: 3511
Total number of 4LAC entries: 3511
Added a total of 3511 redshifts. 1767 had positive values.
 -- From Paolo's catalog -- 
A total of 0 redshifts were correct within 4LAC
Removed a total of 87 redshifts from 4LAC
Added a total of 23 redshifts to 4LAC
Updated a total of 236 redshifts of 4LAC


List of column names: agn_pop_catalog

 ['Source_Name', 'RAJ2000', 'DEJ2000', 'Redshift', 'SpectrumType', 'Pivot_Energy', 'PL_Flux_Density', 'PL_Index', 'LP_Flux_Density', 'LP_Index', 'LP_beta', 'SED_class', 'Variability_Index', 'Frac_Variability', 'Unc_Frac_Variability', 'Flux1000', 'Unc_Flux1000', 'Flux_History', 'Unc_Flux_History']

Number of table rows: 3511


NameError: name 'catalog_with_flux_history' is not defined

NameError: name 'catalog_with_flux_history' is not defined

In [ ]:
catalog_data = agn_pop_catalog['Source_Name', 'SpectrumType', 'Pivot_Energy', 'PL_Flux_Density', 'PL_Index', 'LP_Flux_Density', 'LP_Index', 'LP_beta', 'SED_class', 'Flux1000', 'Unc_Flux1000', 'Frac_Variability', 'Unc_Frac_Variability']

catalog_data

In [ ]:
np.unique(catalog_data['SED_class'])

## Spectrum Flux

## Flux1000 & Unc_Flux1000

We will use 4FGL's values of Flux1000 to compare with the Spectrum Flux we have estimated.


Flux1000 = Integral photon flux from 1 to 100 GeV

Unc Flux1000 = 1$\sigma $ error on integral photon flux from 1 to 100 GeV

In [ ]:
catalog_with_flux1000 = agn_pop_catalog['Source_Name', 'Flux1000', 'Unc_Flux1000'] # 'SED_class'

catalog_with_flux1000

In [ ]:
# ## saving the dataframe as a .csv table

# table_flux1000 = Table.from_pandas(catalog_with_flux1000.to_pandas())
# table_flux1000.write('./resulting_catalogs/flux1000.fits', format='fits', overwrite=True)
# table_flux1000.write('./resulting_catalogs/flux1000.ecsv', format='ascii.ecsv', overwrite=True)

## Upper Limits

$$ \text{spectrum_flux} = \frac{\Sigma_{i} \text{flux}_{i} + n_{UL} \times \text{flux}_{UL}}{n_{i} + n_{UL}} $$

then, we can obtain an estimate of the Upper Limit flux by

$$ \text{flux}_{UL} = \frac{\text{spectrum_flux} \times (n_{i} + n_{UL}) -  \Sigma_{i} \text{flux}_{i}}{n_{UL}} $$

In [ ]:

# ni_flux = 
# n_UL = # number of upper limits
# flux_UL = # UL flux that we want to estimate

# flux_UL = (1/n_UL) * ( spectrum_flux * (ni_flux + n_UL) - flux.sum() )



Adding weights to estimate the mean

In [ ]:
# Exemplo de array de fluxos e pesos
flux = np.array([10, 20, 30, 40, 50])
weights = np.array([1, 2, 1, 2, 1])

# Calculando a média ponderada
weighted_mean = np.average(flux, weights=weights)

# Ou manualmente
weighted_sum = np.sum(flux * weights)
sum_of_weights = np.sum(weights)
weighted_mean_manual = weighted_sum / sum_of_weights

print("Média ponderada:", weighted_mean)
print("Média ponderada (manual):", weighted_mean_manual)


## Comparing cases:

- unc_flux_UL = flux_UL
    flux_UL = 0
    
- $ \text{flux}_{UL} = \frac{\text{spectrum_flux} \times (n_{i} + n_{UL}) -  \Sigma_{i} \text{flux}_{i}}{n_{UL}} $

- Adding weights on UL flux values (bias ?)

## Testing the Fractional Variability Parameter (F_var)

Creating a dataframe with the Fractional Variability parameter per source in the filtered catalog:

In [ ]:
catalog_with_fvar = agn_pop_catalog['Source_Name', 'Frac_Variability', 'Unc_Frac_Variability'] # 'SED_class'

catalog_with_fvar

In [ ]:
## Saving the dataframe as a .csv table

# catalog_with_fvar.to_pandas().to_csv('Fvar_data.csv', index=False)

### Selecting the Flux History of one specific source

Selecting the source Mrk421 = 'J1104.4+3812'

In [ ]:
catalog_with_fvar[catalog_with_fvar['Source_Name'] == 'J1104.4+3812']

We can extract 1-year lightcurve information from the 4FGL:

In [ ]:
catalog_with_flux_history = agn_pop_catalog['Source_Name', 'Flux_History', 'Unc_Flux_History']

catalog_with_flux_history

### Testing the flux selection and Upper Limit Filter

In [ ]:
## Select a single row based on the source name

# 'J2359.2-3134' / 'J1104.4+3812' / 'J0001.2+4741'

selected_source = catalog_with_flux_history[catalog_with_flux_history['Source_Name'] == 'J0001.2+4741']

In [ ]:
selected_source

In [ ]:
## Flux Points

selected_source['Flux_History'][0][:]

In [ ]:
## Unc Flux Points

selected_source['Unc_Flux_History'][0][:]

In [ ]:
## Extracting flux points and errors

## creating a mask for upper limits
mask_source = np.invert(np.isnan(selected_source['Unc_Flux_History'][0][:,0]))

## selecting the values that are not Upper Limits
source_flux = selected_source['Flux_History'][0][mask_source]

## choose the positive uncertainties (seems larger)
source_flux_unc = selected_source['Unc_Flux_History'][0][:,1][mask_source]

In [ ]:
source_flux

In [ ]:
source_flux_unc

In [ ]:
## Testing errors

# fluxes = selected_source["Flux_History"]
# fluxes_unc = selected_source["Unc_Flux_History"][:, 1].copy()
# errors = selected_source["Unc_Flux_History"][0][:]
# nan_locations = np.where(np.isnan(errors[:,0]))[0]

# fluxes_error = []

# for i, err in enumerate(errors):
#     if i not in nan_locations:
#         print(fluxes[i,i])
#         print('err', err)
#         ## selecting only the error bar: flux_error = flux - flux_low_error // flux_high_error - flux
#         flux_point_low_error = fluxes[i,i] - err[0]
#         flux_point_high_error = err[1] - fluxes[i,i]
        
#         if flux_point_high_error >= flux_point_low_error:
#             flux_error = flux_point_high_error
#         else:
#             flux_error = flux_point_low_error
        
#         fluxes_error.append(flux_error)
       
        

## Average of errors
# fluxes_unc[i] = np.average(np.abs(err), axis=0)

## Minimum of errors
# fluxes_unc[i] = np.min(np.abs(err), axis=0)

## Maximum of errors
# fluxes_unc[i] = np.max(np.abs(err), axis=0)
# print(fluxes_unc[i])      

In [ ]:
catalog_with_flux_history

## For different Blazar Classes (Synchrotron for now)

In [ ]:
dataLSP = catalog_data[catalog_data['SED_class'] == 'LSP']
dataISP = catalog_data[catalog_data['SED_class'] == 'ISP']
dataHSP = catalog_data[catalog_data['SED_class'] == 'HSP']

In [ ]:
dataLSP

In [ ]:
print('# of sources in:')

print('\ndataLSP:', len(dataLSP))
print('dataISP:', len(dataISP))
print('dataHSP:', len(dataHSP))

print('\ndataLSP + dataISP + dataHSP:', len(dataLSP) + len(dataISP) + len(dataHSP))
print('catalog_data:', len(catalog_data))
print('are they equal?', len(dataLSP) + len(dataISP) + len(dataHSP) == len(catalog_data))

print('\n--> difference between tables:', len(catalog_data) - (len(dataLSP)+len(dataISP)+len(dataHSP)), 'SOURCES DO NOT HAVE A SED_class REGISTERED')

In [ ]:
# ## Saving the dataframes as .csv tables

# LSP_pandas = dataLSP.to_pandas()
# ISP_pandas = dataISP.to_pandas()
# HSP_pandas = dataHSP.to_pandas()
# whole_data_pandas = catalog_data.to_pandas()

# LSP_pandas.to_csv('LSP_data.csv', index=False)
# ISP_pandas.to_csv('ISP_data.csv', index=False)
# HSP_pandas.to_csv('HSP_data.csv', index=False)
# whole_data_pandas.to_csv('catalog_data.csv', index=False)